In [114]:
import pandas as pd
import os
import difflib
import subprocess

In [115]:
APPROACH = "deepseek-chat"

In [116]:
def collect_cocci_files(path: str, approach: str)->list:
    outputs = []
    for root, dirnames, files in os.walk(path):
        for f in files:
            if f.endswith(f".{approach}.cocci"):
                # f = f.split(".")[:-1]
                # f = ".".join(f)
                outputs.append(
                    {
                        "root": root,
                        "cocci_filename": f
                    }
                )
    return pd.DataFrame(outputs)

In [117]:
PATH = "."
df = collect_cocci_files(PATH, APPROACH)

In [118]:
df.cocci_filename.value_counts()

cocci_filename
final_response_1.deepseek-chat.cocci    14
final_response_0.deepseek-chat.cocci    14
final_response_2.deepseek-chat.cocci    14
Name: count, dtype: int64

In [119]:
def collect_c_files(path: str)->list:
    outputs = []
    for root, dirnames, files in os.walk(path):
        for f in files:
            if f.endswith(f".res.c.sanitized.res.c") and "gpt" not in f and "spinfer" not in f and "deepseek" not in f and "claude" not in f:
                f = f.split(".")[0]
                outputs.append(
                    {
                        "root": root,
                        "c_filename": f
                    }
                )
    return pd.DataFrame(outputs)

In [120]:
df_c = collect_c_files(PATH)
df_c.root.value_counts()

root
./kees_timer1             3
./EXP0-7                  2
./snd_soc                 2
./tcf_block_get-61        2
./dasd_smalloc            2
./sock_poll_wait-84       1
./ttm_bo_init-60          1
./random_ether_addr-84    1
./perf_evlist__mmap-69    1
./uartlite                1
./early_memunmap          1
./dma_pool_alloc-52       1
./free_bootmem-77         1
./tcaction                1
Name: count, dtype: int64

In [121]:
df_c.c_filename.to_list()

['1533533124_2018-08-06_4fd786e6c3d6_qgroup___btrfs_qgroup_release_data',
 '1533533124_2018-08-06_4fd786e6c3d6_disk-io___setup_root',
 '1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc',
 '1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init',
 '1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll',
 '1529689860_2018-06-22_6c1f0a1ffb7c_gemini_gemini_ethernet_port_probe',
 '1524629994_2018-04-25_e4b31b816c47_mt2701-afe-pcm_mt2701_dlm_fe_startup',
 '1524629994_2018-04-25_e4b31b816c47_mtk-afe-fe-dai_mtk_afe_fe_trigger',
 '1519311151_2018-02-22_724daa4fd65d_nouveau_bo_nouveau_bo_new',
 '1360605874_2013-02-11_6d53c3b71d32_uartlite_ulite_get_poll_char',
 '1512266437_2017-12-03_f74b9d3a1ac2_bpf_do_test',
 '1424769208_2015-02-24_8d4a40bc0651_setup_relocate_initrd',
 '1508184939_2017-10-16_b9eaf1872222_fas216_fas216_init',
 '1508184939_2017-10-16_b9eaf1872222_isdn_common_isdn_init',
 '1508184939_2017-10-16_b9eaf1872222_dasd_dasd_alloc_device',
 '1518632953_2018-02-14_ddd05979f89c_bdc_ep_ep_b

In [122]:
for i, row in df_c.sample(n=5, random_state=123).iterrows():
    print(row["root"], row["c_filename"])

./kees_timer1 1508184939_2017-10-16_b9eaf1872222_dasd_dasd_alloc_device
./random_ether_addr-84 1529689860_2018-06-22_6c1f0a1ffb7c_gemini_gemini_ethernet_port_probe
./sock_poll_wait-84 1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll
./tcaction 1534706529_2018-08-19_244cd96adb5f_en_tc_parse_tc_fdb_actions
./ttm_bo_init-60 1519311151_2018-02-22_724daa4fd65d_nouveau_bo_nouveau_bo_new


In [123]:
for i, row in df.sample(n=5, random_state=123).iterrows():
    print(row["root"], row["cocci_filename"])

./kees_timer1 final_response_1.deepseek-chat.cocci
./snd_soc final_response_1.deepseek-chat.cocci
./random_ether_addr-84 final_response_2.deepseek-chat.cocci
./free_bootmem-77 final_response_2.deepseek-chat.cocci
./random_ether_addr-84 final_response_0.deepseek-chat.cocci


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   root            42 non-null     object
 1   cocci_filename  42 non-null     object
dtypes: object(2)
memory usage: 804.0+ bytes


In [125]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   root        20 non-null     object
 1   c_filename  20 non-null     object
dtypes: object(2)
memory usage: 452.0+ bytes


In [126]:
df = pd.merge(df, df_c, on='root', how='inner')
df.drop_duplicates(inplace=True)
df.shape

(60, 3)

In [127]:
df["pred"] = df["cocci_filename"].apply(lambda x: x.split(".")[0])
df["pred"] = df["pred"].apply(lambda x: x.split("_")[-1])
df.head(20)

,root,cocci_filename,c_filename,pred
0,./EXP0-7,final_response_1.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,1
1,./EXP0-7,final_response_1.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,1
2,./EXP0-7,final_response_0.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,0
3,./EXP0-7,final_response_0.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,0
4,./EXP0-7,final_response_2.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,2
5,./EXP0-7,final_response_2.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,2
6,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,1
7,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,1
8,./tcf_block_get-61,final_response_0.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,0
9,./tcf_block_get-61,final_response_0.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,0


# check syntax correctness

In [128]:
def check_syntax_correctness(filepath: str)->bool:
    '''
    Validates a Coccinelle semantic patch by parsing it, writes debug information, and returns a boolean indicating the validity.
    '''
    command = f'spatch --parse-cocci {filepath}'.split()

    try:
        result = subprocess.run(command, check=True, capture_output=True)
        return 1
    except Exception as e:
        print(e)
        return 0

In [129]:
df["is_cocci_valid"] = df.apply(lambda x: check_syntax_correctness(os.path.join(x.root, x.cocci_filename)), axis=1)

Command '['spatch', '--parse-cocci', './tcf_block_get-61/final_response_1.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/final_response_1.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/final_response_0.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/final_response_0.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/final_response_2.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/final_response_2.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './sock_poll_wait-84/final_response_1.deepseek-chat.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './sock_poll_wait-84/final_response_0.

# check how many cocci is successfully applied

In [130]:
df["is_applied"] = df.apply(lambda x:os.path.exists(x.root + "/" + x.c_filename+f".{x.pred}.{APPROACH}.res.c.sanitized.res.c"), axis=1)
df.is_applied.value_counts()

is_applied
True     31
False    29
Name: count, dtype: int64

# compute precision and recall

In [131]:
def open_and_read_file_content(filepath: str)->str:
    """Open the input filepath, then read the content"""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except IOError as e:
        print(f"An error occurred while reading the file: {e}")
    return ""

def remove_unmodified_lines(input_diff: list, is_merge: bool = True)->str:
    additions = []
    removals = []
    for line in input_diff:
        if line.startswith("+") and "+++" not in line:
            additions.append(line)
        elif line.startswith("-") and "---" not in line:
            removals.append(line)
    output_str = removals + additions
    if is_merge:
        return "\n".join(output_str)
    else:
        return removals + additions

def get_diff(file_a: str, file_b: str, n_context: int)->str:
    """Get the diff between file_a and file_b using difflib with n_context length"""
    file_a = open_and_read_file_content(file_a)
    if file_a == "" or "parse error :" in file_a or "init_defs_builtins : " in file_a:
        return ""
    file_b = open_and_read_file_content(file_b)
    if file_b == "" or "parse error :" in file_b or "init_defs_builtins : " in file_b:
        return ""

    lines_a = file_a.splitlines()
    lines_b = file_b.splitlines()        
    # Generate the unified diff
    diff = difflib.unified_diff(
        lines_a,
        lines_b,
        fromfile="initial",
        tofile="final",
        lineterm='',
        n=n_context
    )
    # diff = [x for x in diff if not x.startswith("+++") and not x.startswith("---") and not x.startswith("@@")]
    if n_context == 0:
        diff = remove_unmodified_lines(input_diff=diff, is_merge=False)
    # Convert the diff generator to a single string
    diff_text = '\n'.join(diff)
    return diff_text

In [132]:
df["diff_truth"] = df.apply(lambda x:get_diff(
    file_a = os.path.join(x.root, x.c_filename + ".c.sanitized.c"),
    file_b = os.path.join(x.root, x.c_filename + ".res.c.sanitized.res.c"),
    n_context = 0
), axis=1)

In [133]:
df

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth
0,./EXP0-7,final_response_1.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,1,1,True,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...
1,./EXP0-7,final_response_1.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,1,1,True,- root->objectid = objectid;\n+ root->root_k...
2,./EXP0-7,final_response_0.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,0,1,True,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...
3,./EXP0-7,final_response_0.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,0,1,True,- root->objectid = objectid;\n+ root->root_k...
4,./EXP0-7,final_response_2.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,2,1,True,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...
5,./EXP0-7,final_response_2.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,2,1,True,- root->objectid = objectid;\n+ root->root_k...
6,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,1,0,False,"- err = tcf_block_get(&q->block, &q->filter_l..."
7,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,1,0,False,"- err = tcf_block_get(&q->block, &q->filter_l..."
8,./tcf_block_get-61,final_response_0.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,0,0,False,"- err = tcf_block_get(&q->block, &q->filter_l..."
9,./tcf_block_get-61,final_response_0.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,0,0,False,"- err = tcf_block_get(&q->block, &q->filter_l..."


In [134]:
df["diff_pred"] = df.apply(lambda x:get_diff(
    file_a = os.path.join(x.root, x.c_filename + ".c.sanitized.c"),
    file_b = os.path.join(x.root, x.c_filename + f".{x.pred}.{APPROACH}.res.c.sanitized.res.c"),
    n_context = 0
), axis=1)

Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc.1.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init.1.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc.0.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init.0.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc.2.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init.2.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './sock_poll_wait-84/1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll.1.deepseek-chat.res.c.sanitized.res.c' was no

In [135]:
df["diff_5"] = df.apply(lambda x:get_diff(
    file_a = os.path.join(x.root, x.c_filename + ".c.sanitized.c"),
    file_b = os.path.join(x.root, x.c_filename + f".{x.pred}.{APPROACH}.res.c.sanitized.res.c"),
    n_context = 5
), axis=1)

Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc.1.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init.1.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc.0.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init.0.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr_init_qdisc.2.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb_init.2.deepseek-chat.res.c.sanitized.res.c' was not found.
Error: The file './sock_poll_wait-84/1540294839_2018-10-23_89ab066d4229_tcp_tcp_poll.1.deepseek-chat.res.c.sanitized.res.c' was no

In [136]:
df

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth,diff_pred,diff_5
0,./EXP0-7,final_response_1.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,1,1,True,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...,"--- initial\n+++ final\n@@ -13,11 +13,11 @@\n ..."
1,./EXP0-7,final_response_1.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,1,1,True,- root->objectid = objectid;\n+ root->root_k...,- root->objectid = objectid;\n+ root->root_k...,"--- initial\n+++ final\n@@ -3,11 +3,11 @@\n ..."
2,./EXP0-7,final_response_0.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,0,1,True,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...,"--- initial\n+++ final\n@@ -13,11 +13,11 @@\n ..."
3,./EXP0-7,final_response_0.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,0,1,True,- root->objectid = objectid;\n+ root->root_k...,- root->objectid = objectid;\n+ root->root_k...,"--- initial\n+++ final\n@@ -3,11 +3,11 @@\n ..."
4,./EXP0-7,final_response_2.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,2,1,True,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...,- btrfs_qgroup_free_refroot(BTRFS_I(inode)-...,"--- initial\n+++ final\n@@ -13,11 +13,11 @@\n ..."
5,./EXP0-7,final_response_2.deepseek-chat.cocci,1533533124_2018-08-06_4fd786e6c3d6_disk-io___s...,2,1,True,- root->objectid = objectid;\n+ root->root_k...,- root->objectid = objectid;\n+ root->root_k...,"--- initial\n+++ final\n@@ -3,11 +3,11 @@\n ..."
6,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,1,0,False,"- err = tcf_block_get(&q->block, &q->filter_l...",,
7,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,1,0,False,"- err = tcf_block_get(&q->block, &q->filter_l...",,
8,./tcf_block_get-61,final_response_0.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,0,0,False,"- err = tcf_block_get(&q->block, &q->filter_l...",,
9,./tcf_block_get-61,final_response_0.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,0,0,False,"- err = tcf_block_get(&q->block, &q->filter_l...",,


In [137]:
def compute_precision(truths: list, results: list)->int:
    '''
    Computes the precision of the results compared to the truths, rounding the result to three decimal places.
    '''
    truths_cp = truths.copy()
    is_match = []
    for line in results:
        if line in truths_cp:
            truths_cp.remove(line)
            is_match.append(1)
        else:
            is_match.append(0)
    precision = sum(is_match)/len(is_match) if len(is_match) != 0 else 0
    return round(precision, 3)

In [138]:
df["precision"] = df.apply(lambda x: compute_precision(
    x.diff_truth.splitlines(),
    x.diff_pred.splitlines()
), axis=1)

In [139]:
def compute_recall(truths: list, results: list)->int:
    '''
    Computes the recall of the results compared to the truths, rounding the result to three decimal places.
    '''
    results_cp = results.copy()
    is_match = []
    for line in truths:
        if line in results_cp:
            results_cp.remove(line)
            is_match.append(1)
        else:
            is_match.append(0)
    recall = sum(is_match)/len(is_match) if len(is_match) != 0 else 0
    return round(recall, 3)

In [140]:
df["recall"] = df.apply(lambda x: compute_recall(
    x.diff_truth.splitlines(),
    x.diff_pred.splitlines()
), axis=1)

In [141]:
df["cocci_fullpath"] = df.apply(lambda x: os.path.join(x.root, x.cocci_filename), axis=1)

In [142]:
df.groupby(["root", "cocci_filename", "cocci_fullpath"])[["precision", "recall" ,"is_cocci_valid"]].mean().round(2).reset_index()

,root,cocci_filename,cocci_fullpath,precision,recall,is_cocci_valid
0,./EXP0-7,final_response_0.deepseek-chat.cocci,./EXP0-7/final_response_0.deepseek-chat.cocci,1.00,1.00,1.0
1,./EXP0-7,final_response_1.deepseek-chat.cocci,./EXP0-7/final_response_1.deepseek-chat.cocci,1.00,1.00,1.0
2,./EXP0-7,final_response_2.deepseek-chat.cocci,./EXP0-7/final_response_2.deepseek-chat.cocci,1.00,1.00,1.0
3,./dasd_smalloc,final_response_0.deepseek-chat.cocci,./dasd_smalloc/final_response_0.deepseek-chat....,0.75,0.75,1.0
4,./dasd_smalloc,final_response_1.deepseek-chat.cocci,./dasd_smalloc/final_response_1.deepseek-chat....,0.75,0.75,1.0
5,./dasd_smalloc,final_response_2.deepseek-chat.cocci,./dasd_smalloc/final_response_2.deepseek-chat....,0.00,0.00,0.0
6,./dma_pool_alloc-52,final_response_0.deepseek-chat.cocci,./dma_pool_alloc-52/final_response_0.deepseek-...,0.00,0.00,1.0
7,./dma_pool_alloc-52,final_response_1.deepseek-chat.cocci,./dma_pool_alloc-52/final_response_1.deepseek-...,0.00,0.00,1.0
8,./dma_pool_alloc-52,final_response_2.deepseek-chat.cocci,./dma_pool_alloc-52/final_response_2.deepseek-...,0.00,0.00,1.0
9,./early_memunmap,final_response_0.deepseek-chat.cocci,./early_memunmap/final_response_0.deepseek-cha...,1.00,1.00,1.0


In [143]:
temp = df.groupby(["root", "cocci_filename", "cocci_fullpath"])[["precision", "recall" ,"is_cocci_valid"]].mean().round(2).reset_index()
for index, row in temp.iterrows():
    # Combine root path and filename
    file_path = os.path.join(row['root'], row['cocci_filename'])
    
    # Try to open and print the file contents
    try:
        with open(file_path, 'r') as file:
            print(f"\n--- Contents of file: {file_path} ---")
            print(file.read())
            print("--- End of file ---\n")
    except Exception as e:
        print(f"Error opening file {file_path}: {e}")


--- Contents of file: ./EXP0-7/final_response_0.deepseek-chat.cocci ---
@@
expression root;
@@
- root->objectid
+ root->root_key.objectid
--- End of file ---


--- Contents of file: ./EXP0-7/final_response_1.deepseek-chat.cocci ---
@@
expression root;
expression val;
@@
- root->objectid
+ root->root_key.objectid
@@
expression root;
expression val;
@@
- root->objectid = val
+ root->root_key.objectid = val
--- End of file ---


--- Contents of file: ./EXP0-7/final_response_2.deepseek-chat.cocci ---
@@
expression root;
@@
- root->objectid
+ root->root_key.objectid
--- End of file ---


--- Contents of file: ./dasd_smalloc/final_response_0.deepseek-chat.cocci ---
@@
expression magic, cplength, datasize, device;
@@
- dasd_smalloc_request(magic, cplength, datasize, device)
+ dasd_smalloc_request(magic, cplength, datasize, device, NULL)
--- End of file ---


--- Contents of file: ./dasd_smalloc/final_response_1.deepseek-chat.cocci ---
@@
expression magic, cplength, datasize, device;
@@
- das

# inspect the best patch for each cluster then combine them

In [144]:
best_patch = [
f"./EXP0-7/final_response_0.{APPROACH}.cocci",
f"./dasd_smalloc/final_response_0.{APPROACH}.cocci",
f"./dma_pool_alloc-52/final_response_0.{APPROACH}.cocci",
f"./early_memunmap/final_response_0.{APPROACH}.cocci",
f"./free_bootmem-77/final_response_0.{APPROACH}.cocci",
f"./kees_timer1/final_response_1.{APPROACH}.cocci",
f"./perf_evlist__mmap-69/final_response_0.{APPROACH}.cocci",
f"./random_ether_addr-84/final_response_0.{APPROACH}.cocci",
f"./snd_soc/final_response_0.{APPROACH}.cocci",
f"./sock_poll_wait-84/final_response_2.{APPROACH}.cocci",
f"./tcaction/final_response_0.{APPROACH}.cocci",
f"./tcf_block_get-61/final_response_0.{APPROACH}.cocci",
f"./ttm_bo_init-60/final_response_2.{APPROACH}.cocci",
f"./uartlite/final_response_0.{APPROACH}.cocci"
]

In [145]:
len(best_patch)

14

In [146]:
df.root.unique()

array(['./EXP0-7', './tcf_block_get-61', './sock_poll_wait-84',
       './random_ether_addr-84', './snd_soc', './ttm_bo_init-60',
       './uartlite', './perf_evlist__mmap-69', './early_memunmap',
       './kees_timer1', './dma_pool_alloc-52', './free_bootmem-77',
       './tcaction', './dasd_smalloc'], dtype=object)

In [147]:
df[~df.root.isin(df[df.cocci_fullpath.isin(best_patch)].root.unique())].root.unique()

array([], dtype=object)

In [148]:
df[df.cocci_fullpath.isin(best_patch)].shape

(20, 12)

In [149]:
import re
# get multiple semantic patches into 1 list to be merged
folder_to_sp = {}
for path in best_patch:
    folder = path.split("/")[1]
    pattern = r"_c[0-9]+"

    if folder not in folder_to_sp:
        folder_to_sp[folder] = []
    
    with open(path, "r") as f:
        sp = f.read()
    
    folder_to_sp[folder].append(sp)

In [150]:
folder_to_sp

{'EXP0-7': ['@@\nexpression root;\n@@\n- root->objectid\n+ root->root_key.objectid'],
 'dasd_smalloc': ['@@\nexpression magic, cplength, datasize, device;\n@@\n- dasd_smalloc_request(magic, cplength, datasize, device)\n+ dasd_smalloc_request(magic, cplength, datasize, device, NULL)'],
 'dma_pool_alloc-52': ['@@\nexpression pool, flags, dma;\nidentifier table;\nexpression size;\n@@\n- table->start_bd = dma_pool_alloc(pool, flags, dma);\n+ table->start_bd = dma_pool_zalloc(pool, flags, dma);\n  if (!table->start_bd)\n    {\n      ...\n    }\n- memset(table->start_bd, 0, size);'],
 'early_memunmap': ['@@\nexpression addr, size;\n@@\n- early_iounmap(addr, size)\n+ early_memunmap(addr, size)'],
 'free_bootmem-77': ['@@\nexpression e1, e2;\n@@\n- free_bootmem(e1, e2)\n+ memblock_free(e1, e2)'],
 'kees_timer1': ['@@\nexpression timer, func;\nexpression data;\n@@\n- init_timer(&timer);\n- timer.function = func;\n- timer.data = (unsigned long)data;\n+ setup_timer(&timer, func, (unsigned long)da

In [151]:
import shutil
import re

TEST_PATH = "../test/"
FILENAME = f"sp_out.final.{APPROACH}.cocci"
for directory, patches in folder_to_sp.items():
    combined_patch = ""
    for p in patches:
        combined_patch += p
        combined_patch += "\n\n\n"

    # source_filepath = os.path.join(".", directory, FILENAME)
    pattern = r"_c[0-9]+"
    matches = re.findall(pattern, directory)
    if matches:
        directory = directory.split("_")[:-1]
        directory = "_".join(directory)

    target_filepath = os.path.join(TEST_PATH, directory, FILENAME) 
    with open(target_filepath, "w") as f:
        f.write(combined_patch)
    print(f"Saved the final patch to: {target_filepath}")
    # shutil.copy(source_filepath, target_filepath)

Saved the final patch to: ../test/EXP0-7/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/dasd_smalloc/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/dma_pool_alloc-52/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/early_memunmap/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/free_bootmem-77/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/kees_timer1/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/perf_evlist__mmap-69/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/random_ether_addr-84/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/snd_soc/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/sock_poll_wait-84/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/tcaction/sp_out.final.deepseek-chat.cocci
Saved the final patch to: ../test/tcf_block_get-61/sp_out.final.deepseek-chat.cocci
Saved the final patch

# check final example coverage

- Here, we only care about the precision and recall
    - If tie, then choose the highest one


- the selection of the best patch in the previous step guarantee that the patch will also yield similar result as the choosing the highest precision and recall 
    - note that we also use precision and recall as one of the criterion when selecting the best patch
    - but, the selection process is more refined (i check the overfitting issue)

In [152]:
df.c_filename.nunique()

20

In [153]:
df['rank_score'] = (df['precision'] + df['recall'] + df["is_cocci_valid"]) / 3
best_preds_per_cluster = df.loc[df.groupby(['c_filename'])['rank_score'].idxmax()]
best_clusters = best_preds_per_cluster.loc[
    best_preds_per_cluster.groupby('c_filename')['rank_score'].idxmax()
]
best_clusters = best_clusters.drop(columns=['rank_score'])

In [154]:
best_clusters.shape

(20, 12)

In [155]:
best_clusters

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth,diff_pred,diff_5,precision,recall,cocci_fullpath
27,./uartlite,final_response_1.deepseek-chat.cocci,1360605874_2013-02-11_6d53c3b71d32_uartlite_ul...,1,1,True,- if (!(ioread32be(port->membase + ULITE_STAT...,- if (!(ioread32be(port->membase + ULITE_STAT...,"--- initial\n+++ final\n@@ -1,6 +1,6 @@\n stat...",1.0,1.0,./uartlite/final_response_1.deepseek-chat.cocci
33,./early_memunmap,final_response_1.deepseek-chat.cocci,1424769208_2015-02-24_8d4a40bc0651_setup_reloc...,1,1,True,"- early_iounmap(p, clen + slop);\n+ ...","- early_iounmap(p, clen + slop);\n+ ...","--- initial\n+++ final\n@@ -24,11 +24,11 @@\n ...",1.0,1.0,./early_memunmap/final_response_1.deepseek-cha...
6,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1507896057_2017-10-13_69d78ef25c7b_sch_drr_drr...,1,0,False,"- err = tcf_block_get(&q->block, &q->filter_l...",,,0.0,0.0,./tcf_block_get-61/final_response_1.deepseek-c...
38,./kees_timer1,final_response_1.deepseek-chat.cocci,1508184939_2017-10-16_b9eaf1872222_dasd_dasd_a...,1,0,False,- init_timer(&device->timer);\n- device->tim...,,,0.0,0.0,./kees_timer1/final_response_1.deepseek-chat.c...
36,./kees_timer1,final_response_1.deepseek-chat.cocci,1508184939_2017-10-16_b9eaf1872222_fas216_fas2...,1,0,False,- init_timer(&info->eh_timer);\n- info->eh_t...,,,0.0,0.0,./kees_timer1/final_response_1.deepseek-chat.c...
37,./kees_timer1,final_response_1.deepseek-chat.cocci,1508184939_2017-10-16_b9eaf1872222_isdn_common...,1,0,False,- init_timer(&dev->timer);\n- dev->timer.fun...,,,0.0,0.0,./kees_timer1/final_response_1.deepseek-chat.c...
30,./perf_evlist__mmap-69,final_response_1.deepseek-chat.cocci,1512266437_2017-12-03_f74b9d3a1ac2_bpf_do_test,1,1,True,"- err = perf_evlist__mmap(evlist, opts.mmap_p...","- err = perf_evlist__mmap(evlist, opts.mmap_p...","--- initial\n+++ final\n@@ -49,11 +49,11 @@\n ...",1.0,1.0,./perf_evlist__mmap-69/final_response_1.deepse...
7,./tcf_block_get-61,final_response_1.deepseek-chat.cocci,1513791319_2017-12-20_8d1a77f974ca_sch_htb_htb...,1,0,False,"- err = tcf_block_get(&q->block, &q->filter_l...",,,0.0,0.0,./tcf_block_get-61/final_response_1.deepseek-c...
45,./dma_pool_alloc-52,final_response_1.deepseek-chat.cocci,1518632953_2018-02-14_ddd05979f89c_bdc_ep_ep_b...,1,1,True,- bd_table->start_bd = dma_pool_alloc(bdc...,,,0.0,0.0,./dma_pool_alloc-52/final_response_1.deepseek-...
26,./ttm_bo_init-60,final_response_2.deepseek-chat.cocci,1519311151_2018-02-22_724daa4fd65d_nouveau_bo_...,2,1,True,"- ret = ttm_bo_init(&drm->ttm.bdev, &nvbo->bo...","- ret = ttm_bo_init(&drm->ttm.bdev, &nvbo->bo...","--- initial\n+++ final\n@@ -82,11 +82,11 @@\n ...",1.0,1.0,./ttm_bo_init-60/final_response_2.deepseek-cha...


In [156]:
round(best_clusters.precision.mean(), 2), round(best_clusters.recall.mean(), 2), round(best_clusters.drop_duplicates(subset=["root"]).is_cocci_valid.mean(), 2)

(np.float64(0.52), np.float64(0.52), np.float64(0.71))

In [157]:
mean_df = best_clusters.groupby(["root"])[["is_cocci_valid", "precision", "recall"]].mean().round(2)
mean_df

,is_cocci_valid,precision,recall
root,,,
./EXP0-7,1.0,1.00,1.00
./dasd_smalloc,1.0,0.75,0.75
./dma_pool_alloc-52,1.0,0.00,0.00
./early_memunmap,1.0,1.00,1.00
./free_bootmem-77,1.0,1.00,1.00
./kees_timer1,0.0,0.00,0.00
./perf_evlist__mmap-69,1.0,1.00,1.00
./random_ether_addr-84,1.0,1.00,1.00
./snd_soc,0.0,0.00,0.00


In [158]:
# Step 1: Group the data by 'root' and 'cocci_filename'
grouped = best_clusters.groupby(['root'])

# Step 2: Compute total samples per group
group_counts = grouped.size().reset_index(name='num_samples')
temp_df = best_clusters.merge(group_counts, on=['root'])

total_samples = temp_df['num_samples'].sum()
weighted_precision = (temp_df['precision'] * temp_df['num_samples']).sum() / total_samples
weighted_recall = (temp_df['recall'] * temp_df['num_samples']).sum() / total_samples


weighted_precision, weighted_recall

(np.float64(0.4117647058823529), np.float64(0.4117647058823529))

In [159]:
mean_df.reset_index(inplace=True)
mean_df["root"] = mean_df["root"].apply(lambda x: x.split("/")[-1])
mean_df.to_csv("result_perdir_deepseek.csv", index=False)